In [2]:
import time
import serial
import random
import math

In [3]:
freqstartoffset = 512
freqstopoffset = 1024
gainoffset = 1536
loadoffset = 2048
moveoffset = 2560
writeStart = bytearray([161,0,0,0,0,0,1])
writeStop = bytearray([161,0,1,0,0,0,1])

In [4]:
def getFTW(freq):
    return int(round(freq*2**28/(800.0/3.0)))

def writestartfreq(freq,channel):
    addr = freqstartoffset + channel
    FTW = getFTW(freq*2.0)
      
    addr_hi = int(math.floor(addr / 256))
    addr_lo = addr - addr_hi * 256
    
    FTW3 = int(math.floor(FTW / 256 / 256 / 256))
    FTW = FTW - FTW3 * 256 * 256 * 256
    FTW2 = int(math.floor(FTW / 256 / 256))
    FTW = FTW - FTW2 * 256 * 256
    FTW1 = int(math.floor(FTW / 256))
    FTW = FTW - FTW1 * 256
    FTW0 = int(math.floor(FTW))
    return bytearray([161,addr_hi, addr_lo,FTW3,FTW2,FTW1,FTW0])

def writestopfreq(freq,channel):
    addr = freqstopoffset + channel
    FTW = getFTW(freq*2.0)
    
    addr_hi = int(math.floor(addr / 256))
    addr_lo = addr - addr_hi * 256
    
    FTW3 = int(math.floor(FTW / 256 / 256 / 256))
    FTW = FTW - FTW3 * 256 * 256 * 256
    FTW2 = int(math.floor(FTW / 256 / 256))
    FTW = FTW - FTW2 * 256 * 256
    FTW1 = int(math.floor(FTW / 256))
    FTW = FTW - FTW1 * 256
    FTW0 = int(math.floor(FTW))
    return bytearray([161,addr_hi, addr_lo,FTW3,FTW2,FTW1,FTW0])

def writegain(gain,channel):
    addr = gainoffset + channel
    
    addr_hi = int(math.floor(addr / 256.0))
    addr_lo = addr - addr_hi * 256
    
    GW3, GW2 = 0, 0
    GW1 = int(math.floor(gain / 256.0))
    gain = gain - GW1 * 256
    GW0 = int(math.floor(gain))
    
    return bytearray([161,addr_hi, addr_lo,GW3,GW2,GW1,GW0])

def writeloadphase(phase,channel):
    addr = loadoffset + channel
    
    addr_hi = int(math.floor(addr / 256.0))
    addr_lo = addr - addr_hi * 256
    
    GW3, GW2 = 0, 0
    GW1 = int(math.floor(phase / 256.0))
    phase = phase - GW1 * 256
    GW0 = int(math.floor(phase))
    
    return bytearray([161,addr_hi, addr_lo,GW3,GW2,GW1,GW0])

def writemovephase(ser, phase,channel):
    addr = moveoffset + channel
    
    addr_hi = int(math.floor(addr / 256.0))
    addr_lo = addr - addr_hi * 256
    
    GW3 = 0
    GW2 = 0
    GW1 = int(math.floor(phase / 256.0))
    phase = phase - GW1 * 256
    GW0 = int(math.floor(phase))
    
    return bytearray([161,addr_hi, addr_lo,GW3,GW2,GW1,GW0])

def writeOnOff(ser, onoff):
    onoff3 = int(math.floor(onoff / 256 / 256 / 256))
    onoff = onoff - onoff3 * 256 * 256 * 256
    onoff2 = int(math.floor(onoff / 256 / 256))
    onoff = onoff - onoff2 * 256 * 256
    onoff1 = int(math.floor(onoff / 256))
    onoff = onoff - onoff1 * 256
    onoff0 = int(math.floor(onoff))
    serWrite(ser, bytearray([161,0, 3,onoff3,onoff2,onoff1,onoff0]))

In [5]:
def serWrite(serObj, msg):
    print('.',end='')
    if serObj.write(msg) != len(msg):
        raise RuntimeError("Serial write failed! Didn't write all of msg!")

# Auxiliary Tweezer Control: Run Here

### turn off

In [103]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 0)

.

### turn on

In [42]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 1)

.

### settings

### 1 tone...

In [228]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 0)

.

In [227]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 2**2-1)

.

In [205]:
freq = 90
gain = 38000

In [206]:
with serial.Serial('COM3', 77700) as ser:
    print ('serial port = ' + ser.name)
    print('.', end='')
    ser.write(writegain(gain,0))
    ser.write(writestartfreq(freq,0))
    ser.write(writestopfreq(freq,0))
    ser.write(writeloadphase(0,0))
    ser.write(writegain(gain,1))
    ser.write(writestartfreq(freq,1))
    ser.write(writestopfreq(2,1))
    ser.write(writeloadphase(0,1))

serial port = COM3
.

### 2 Tones

In [181]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 0)

.

In [182]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 2**2-1)

.

In [168]:
freq = 90 - 0.5684288905435593
f_s = 0.5684288905435593*2
gain = 52000

In [169]:
with serial.Serial('COM3', 77700) as ser:
    print ('serial port = ' + ser.name)
    for i in range(2):
        print('.', end='')
        ser.write(writegain(gain,i))
        ser.write(writestartfreq(freq+i*f_s,i))
        ser.write(writestopfreq(2,i))
    for i, phase in enumerate(p_d):        
        ser.write(writeloadphase(phase,i))

serial port = COM3
..

In [139]:
ah.maximizeAomPerformance(90, 90, 0.5684288905435593*2, 2, 2);

horFreqs [89.43157110945644, 90.56842889054356]
horAmps [1. 1.]
Hor-Phases: ['0.01523974781', '0.000000000']
vertFreqs [89.43157110945644, 90.56842889054356]
vertAmps [1. 1.]
Vert-Phases: ['0.2631166088', '0.000000000']


In [166]:
p = ['0.01523974781', '0.000000000']
p = [float(x) for x in p]
p_d = [get12BitPhase(x) for x in p]

In [167]:
p_d

[10, 0]

### 5 tones

In [136]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 0)

.

In [135]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 2**5-1)

.

In [131]:
freq = 90 - 4 * 0.5684288905435593
f_s = 0.5684288905435593*2
gain = 32000

In [133]:
-27+50

23

In [132]:
with serial.Serial('COM3', 77700) as ser:
    print ('serial port = ' + ser.name)
    for i in range(5):
        print('.', end='')
        ser.write(writegain(gain,i))
        ser.write(writestartfreq(freq+i*f_s,i))
        ser.write(writestopfreq(2,i))
    for i, phase in enumerate(p_d):        
        ser.write(writeloadphase(phase,i))

serial port = COM3
.....

In [121]:
ah.maximizeAomPerformance(90, 90, 0.5684288905435593*2, 5, 5);

horFreqs [87.72628443782577, 88.86314221891288, 90.0, 91.13685778108713, 92.27371556217425]
horAmps [1. 1. 1. 1. 1.]
Hor-Phases: ['0.2673006673', '0.3479054088', '2.237734896', '5.936527978', '0.000000000']
vertFreqs [87.72628443782577, 88.86314221891288, 90.0, 91.13685778108713, 92.27371556217425]
vertAmps [1. 1. 1. 1. 1.]
Vert-Phases: ['0.5140596388', '0.4753336414', '2.245931902', '5.825411787', '0.000000000']


In [59]:
import math as m
import numpy as np

In [122]:
p = ['0.2673006673', '0.3479054088', '2.237734896', '5.936527978', '0.000000000']
p = [float(x) for x in p]
p_d = [get12BitPhase(x) for x in p]

In [123]:
p_d

[174, 227, 1459, 3870, 0]

### 10 tones

In [90]:
with serial.Serial('COM3', 77700) as ser:
    writeOnOff(ser, 2**10-1)

.

In [77]:
freq = 85.5
gain = 25000

In [78]:
with serial.Serial('COM3', 77700) as ser:
    print ('serial port = ' + ser.name)
    for i in range(10):
        print('.', end='')
        ser.write(writegain(gain,i))
        ser.write(writestartfreq(freq+i,i))
        ser.write(writestopfreq(2,i))
    ser.write(writeloadphase(796 ,1))
    ser.write(writeloadphase(53 ,2))
    ser.write(writeloadphase(1566 ,3))
    ser.write(writeloadphase(2223 ,4))
    ser.write(writeloadphase(3090 ,5))
    ser.write(writeloadphase(4075 ,6))
    ser.write(writeloadphase(2879 ,7))
    ser.write(writeloadphase(3480 ,8))
    ser.write(writeloadphase(2258 ,9))
    ser.write(writeloadphase(0 ,10))

serial port = COM3
..........

In [66]:
import AnalysisHelpers as ah

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [68]:
ah.maximizeAomPerformance(90, 90, 1, 10, 10);

horFreqs [85.5, 86.5, 87.5, 88.5, 89.5, 90.5, 91.5, 92.5, 93.5, 94.5]
horAmps [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Hor-Phases: ['1.221648960', '0.08121443370', '2.402351112', '3.409803034', '4.739306867', '6.251334488', '4.417023833', '5.337870099', '3.464335017', '0.000000000']
vertFreqs [85.5, 86.5, 87.5, 88.5, 89.5, 90.5, 91.5, 92.5, 93.5, 94.5]
vertAmps [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Vert-Phases: ['1.638022210', '0.04944901896', '2.734469931', '2.875436793', '4.984842304', '6.079892693', '4.619504593', '5.473911521', '3.534497763', '0.000000000']


In [50]:
2**12

4096

In [59]:
import math as m
import numpy as np

In [57]:
def get12BitPhase(phase_radians):
    """
    The fpga has 12 bit phase resolution. The number to send to the fpga is a
    12 bit integer (0-4095) which is normalized to 2*pi.
    This function converts a phase in radians to the 12-bit fpga units.
    
    phase_radians: the desired phase, in radians
    return: the phase converted to the units to write directly to the fpga.
    """
    return round((phase_radians / (2*np.pi)) * 2**12)
    

In [74]:
p = ['1.221648960', '0.08121443370', '2.402351112', '3.409803034', '4.739306867', 
     '6.251334488', '4.417023833', '5.337870099', '3.464335017', '0.000000000']
p = [float(x) for x in p]
p_d = [get12BitPhase(x) for x in p]

In [75]:
p_d

[796, 53, 1566, 2223, 3090, 4075, 2879, 3480, 2258, 0]

## scrap